## Debugging q Code

First part of this chapter we make a brief overview of the native errors we may encounter when using incorrect syntax.

Then we focus on how to debug the code when encountering errors and move on to advise on how we can structure our codebase better.

### 5.1 Introduction to making it wrong: Errors

2 main pain points:

    - Avoiding simple syntax errors when initially writing our code.
    - interpreting the generally obscure and short q errors and how we can make these more verbose while our code increases in complexity
    

*syntax error*

    - May waiste time to debug a syntax error like missing semi colon. Modern IDEs have plugins like intelliJ or QPad for q. Still the console provides an interactive way to run and thereby test the code.
    
*Runtime Errors*

    - An error signal, or what is known as an exception, is denoted in q with a single quote folowed by a word or phrase signaling the error type.

In [1]:
wrongFunction:{[x;y] add:x+y; add:add + 50; z+3};

In [1]:
wrongFunction[50;12]

[0;31mz[0m: [0;31mz[0m

The error returns z (note there is no single quote due to the jupyter notebook ide), which is not defined. It also shows the location of the error in the lines [1] and [0].

More frequently we find the error to be a build in q error.

Full list of errors:
  - https://code.kx.com/q/basics/errors/
  - https://github.com/prodrive11/log4q


In [2]:
// runs as intended
two:2;
two xexp 3

8f


In [2]:
//type error
three xexp 3

[0;31mthree[0m: [0;31mthree[0m

q is strongly types. Partially updating values of a variable to a different type results in a 'type error

In [2]:
show prices:101 101.5 101.3;
prices,: 10

101 101.5 101.3


[0;31mtype[0m: [0;31mtype[0m

**NOTE: commonly causes a hard to find bug**
  - when we forget to end a like with a semicolon, we may accidentially merge two statements which were meant to be processed indipendently.
  - Due to the commonality of this error, two ways to minimise are:
      - type check for function arguments
      - logging to trace variables passed into functions

In [3]:
.quantQ.myspace.myfunc:{[prices;newPrices]
    //prices -- verctor of prices
    if[type[prices] <> type[newPrices]; '"Prices are incompatable"];
    : prices,newPrices }

In [4]:
.quantQ.myspace.myfunc[1 2 3f; 10 20 30f]

1 2 3 10 20 30f


In [4]:
.quantQ.myspace.myfunc[1 2 3; 10.2 11.1 12.2f]

[0;31mPrices are incompatable[0m: [0;31mPrices are incompatable[0m

Notice the error message stated in the function is returned.

'lenght - Another common error type appearns when we perform an operation on two object5s which have incompatible lenghts:

In [4]:
1 2 + 1 2 3

[0;31mlength[0m: [0;31mlength[0m

'mismatch - When two tables are being joined with an upsert (or a comma join) and the columns between them differ.

In [4]:
t1:([]a:1 2;b:3 4);
t2:([]c:10 20 30);
t1,t2

[0;31mmismatch[0m: [0;31mmismatch[0m

In [4]:
t1 upsert t2

[0;31mmismatch[0m: [0;31mmismatch[0m

#### ERRORS

'constants - is an error for when we have declared too many constants, more than 96. This is q's guide to refactor our code into funcitons, and if in functions, break it into smaller functions.

'domain - is an error for when we evaluate a function outside of the natural domain.

'from - is an error for when we use one of the sql commands and forget (or misspell) the statement from or omit the table name.

'noupdate - is an error for when we work on a server which has protection aginst writing or updating globals.

'nyi - is an error for when a command looks meaningful but not able to exicute. Stands for 'not yet implimented'.

'os - is an error for when the operating system command produces an error during its execution.

'params - is an error for when a function with more than eight parameters is constructed.

'rank - is an error for when the rank/valeance is lower than how its used. For example calling a function with more variables than the function expects.

'wsfull - is an error for when we run out of virtual memory.

#### 5.2 Debugging the code

Lets look at the tools avilable to debug. Lets look again at the wrongFunction example

In [5]:
\l test/test.q

In [6]:
wrongFunction

{[x;y]
    add:x+y;
    add:add + 50;
    add:add + z}


In [6]:
wrongFunction[1;2]

[0;31mz[0m: [0;31mz[0m

you can see that the 